In [1]:
from __future__ import print_function
import torch
from torch import nn
from torch import optim
from torch.autograd import Variable
import os

import numpy as np

from torch_model import TextRNN, TextCNN
from cnews_loader import read_vocab, read_category, batch_iter, process_file, build_vocab

2023-12-13 15:59:53.413257: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-13 15:59:53.588539: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-13 15:59:53.592336: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/nvidia/lib:/usr

In [2]:
base_dir = 'cnews'
train_dir = os.path.join(base_dir, 'train.txt')
test_dir = os.path.join(base_dir, 'test.txt')
val_dir = os.path.join(base_dir, 'dev.txt')
vocab_dir = os.path.join(base_dir, 'vocab.txt')

pad_len = 32

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def evaluate(model, Loss, x_val, y_val):
    """测试集上准确率评估"""
    batch_val = batch_iter(x_val, y_val, 64)
    acc = 0
    los = 0
    for x_batch, y_batch in batch_val:
        size = len(x_batch)
        x = np.array(x_batch)
        y = np.array(y_batch)
        x = torch.LongTensor(x).to(device)
        y = torch.Tensor(y).to(device)
        # y = torch.LongTensor(y)
        # x = Variable(x)
        # y = Variable(y)
        out = model(x)
        loss = Loss(out, y)
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
        loss_value = np.mean(loss.cpu().detach().numpy())
        accracy = np.mean((torch.argmax(out, 1) == torch.argmax(y, 1)).cpu().numpy())
        acc +=accracy*size
        los +=loss_value*size
    return los/len(x_val), acc/len(x_val)

In [4]:
def train():
    x_train, y_train = process_file(train_dir, word_to_id, cat_to_id,pad_len)#获取训练数据每个字的id和对应标签的oe-hot形式
    x_val, y_val = process_file(val_dir, word_to_id, cat_to_id,pad_len)
    #使用LSTM或者CNN
    # model = TextRNN()
    model = TextCNN()
    
    model.to(device)
    
    #选择损失函数
    Loss = nn.MultiLabelSoftMarginLoss()
    # Loss = nn.BCELoss()
    # Loss = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(),lr=0.001)
    best_val_acc = 0
    for epoch in range(10):
        i = 0
        print('epoch:{}'.format(epoch))
        batch_train = batch_iter(x_train, y_train,64)
        for x_batch, y_batch in batch_train:
            i +=1
            # print(i)
            x = np.array(x_batch)
            y = np.array(y_batch)
            x = torch.LongTensor(x).to(device)
            y = torch.Tensor(y).to(device)
            # y = torch.LongTensor(y)
            # x = Variable(x)
            # y = Variable(y)
            out = model(x)
            loss = Loss(out,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # 对模型进行验证
            if i % 300 == 0:
                los, accracy = evaluate(model, Loss, x_val, y_val)
                print('loss:{},accracy:{}'.format(los, accracy))
                if accracy > best_val_acc:
                    torch.save(model.state_dict(), 'model_params.pkl')
                    best_val_acc = accracy

In [5]:
#获取文本的类别及其对应id的字典
categories, cat_to_id = read_category()
#获取训练文本中所有出现过的字及其所对应的id
build_vocab(train_dir, vocab_dir)
words, word_to_id = read_vocab(vocab_dir)
#获取字数
vocab_size = len(words)
print('train')
train()

train
epoch:0
loss:0.2864182671070099,accracy:0.3294
loss:0.24473629155158996,accracy:0.4607
loss:0.22387070829868316,accracy:0.529
loss:0.20646047756671906,accracy:0.5865
loss:0.1948151601791382,accracy:0.6224
loss:0.18260347797870635,accracy:0.6568
loss:0.17390141050815583,accracy:0.6797
loss:0.16504904317855834,accracy:0.7038
loss:0.1612353448152542,accracy:0.7165
epoch:1
loss:0.15258612034320831,accracy:0.7308
loss:0.15020932745933532,accracy:0.737
loss:0.14543523077964782,accracy:0.7521
loss:0.14237202477455138,accracy:0.7556
loss:0.1391454796552658,accracy:0.7629
loss:0.1371999620437622,accracy:0.7641
loss:0.13539794983863832,accracy:0.7713
loss:0.1338011112332344,accracy:0.7741
loss:0.13123095686435698,accracy:0.7777
epoch:2
loss:0.1280719792366028,accracy:0.7821
loss:0.12920602531433106,accracy:0.7805
loss:0.12630194010734558,accracy:0.7887
loss:0.12476422098875045,accracy:0.7926
loss:0.12467464599609375,accracy:0.7937
loss:0.12458377424478531,accracy:0.7934
loss:0.122614682817